In [ ]:
import boto3
import winsound
import time
import numpy as np
import cv2
import requests
from IPython.display import clear_output
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

def detect_text(photo):

    client = boto3.client('rekognition')

    image = Image.open(photo)

    stream = io.BytesIO()
    image.save(stream,format="JPEG")
    image_binary = stream.getvalue()

    response = client.detect_text(Image={'Bytes':image_binary})
                        
    textDetections=response['TextDetections']
    c=0
    for text in textDetections:
        if text['Type']=='WORD':
            c = c+1
    return c,response,image

def main(): 

    # 5 secs Timer to prepare
    for i in range(5,0,-1):
        time.sleep(1)
        clear_output()
        if i == 1:
            print('GO!')
            winsound.Beep(800,900)
        else:
            print (i)
    print('Keep writing')
    start_time = time.time()
    time.sleep(15)

    print('Okay you can stop writing now')

    winsound.Beep(300,900)
    

    elapsed_time = time.time() - start_time
    
    # Connects to your computer's default camera
    url='http://192.168.43.1:8080/shot.jpg'
    # Capture frame
    img_resp = requests.get(url,verify=False)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    frame = cv2.imdecode(img_arr, -1)

    #save image as 'new.jpg'
    cv2.imwrite('new.jpg',frame)
    
    photo = 'new.jpg'
    
    text_count,res,image =detect_text(photo)


    wpm = (text_count*4)

    print(f'elapsed_time :- {elapsed_time}')
    print(f'words written :- {text_count}')

    #Put wordsPerMinute here
    print(f'Your speed is {wpm} words per min')
    
    #if you want to see how well it did in detecting those words
    choice = input('Want to see the words counted (y/n)?')
    if choice == 'y':
        textDetections=res['TextDetections']
        
        imgWidth, imgHeight = image.size  
        draw = ImageDraw.Draw(image)

        # Calculate and display a bounding box around each detected word
        for textDetail in res['TextDetections']:

            box = textDetail['Geometry']['BoundingBox']
            imgWidth, imgHeight = image.size
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']

            points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top)

            )
            draw.line(points, fill='#00d400', width=5)

        image.show()



if __name__ == "__main__":
    main()